Есть две идеи:
### 1-ая идея:

- Обучение:
Вытащить из корпуса все positive / negative слова
Векторизовать каждое слово с помощью word2vec
Написать простой классификатор (вектор - класс)

- Тестирование:
На вход поступает слово
Вектурезуем его с помощью word2vec и делаем predict


### 2-ая идея

- Обучение
Подать на word2vec сначала негативные слова, векторизовать, записать в массив. 
Потом подать на word2vec позитивные, векторизовать. 
Получится два облака.

- Тестирование
Далее входные слова сравнивать с каждым из этих двух облаков.
Берем входное слово, сравниваем его с каждым словом из позитивного облака, запиисываем расстояние. 
Потом сравниваем с каждым словом из негативного облака и тоже считаем расстояние. 
Сравниваем результаты по расстояниям по разным облакам.

In [1]:
# -*- coding: utf-8 -*-
from __future__ import division
import re
import random
import pandas as pd
import numpy as np
from lxml import etree

import pymorphy2
import math
from nltk.tokenize import TreebankWordTokenizer
from stop_words import get_stop_words

from text_cleanisation_NLP import text_to_words
import gensim

import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn import cross_validation

morph = pymorphy2.MorphAnalyzer()
tokenizer = TreebankWordTokenizer()

RUS_LETTERS = u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def parse_xml(filename):
    with open(filename, encoding='utf-8') as f:
        xml = f.read()

    dict = {}
    text = []
    category = []
    sentiment = []
    term = []

    root = etree.fromstring(xml)
    for child in root:
        for aspect in child[3]:
            if aspect.attrib['type'] == 'implicit' and aspect.attrib['sentiment']!= 'both' and aspect.attrib['sentiment']!= 'neutral':
                text.append(child[2].text)
                category.append(aspect.attrib['category'])
                sentiment.append(aspect.attrib['sentiment'])
                term.append(aspect.attrib['term'])

    dict['text'] = text
    dict['category'] = category
    dict['sentiment'] = sentiment
    dict['term'] = term

    
    return dict

In [3]:
text_train = parse_xml('SentiRuEval_rest_markup_train.xml')
text_test = parse_xml('SentiRuEval_rest_markup_test.xml')

In [4]:
# Создаем датафрейм из тестового и тренировочного корпуса

df1 = pd.DataFrame(text_train)
df2 = pd.DataFrame(text_test)

frames = [df1, df2]
df = pd.concat(frames)

In [5]:
df.head()

,category,sentiment,term,text
0,Food,positive,вкусное,"День 8-го марта прошёл, можно и итоги подвести..."
1,Interior,positive,уютный,"День 8-го марта прошёл, можно и итоги подвести..."
2,Food,positive,вкусные,Отмечали в этом ресторане день рождение на пер...
3,Interior,positive,уютно,Отмечали в этом ресторане день рождение на пер...
4,Interior,positive,красиво,Отмечали в этом ресторане день рождение на пер...


In [6]:
df.describe()

,category,sentiment,term,text
count,1367,1367,1367,1367
unique,5,2,505,371
top,Food,positive,вкусно,Отмечали свадьбу 15 марта. Ах-Ах!!! Замечатель...
freq,538,1235,137,14


In [7]:
df.groupby('sentiment').describe()

category    term  \
sentiment                           
negative  count       132     132   
          unique        5     105   
          top        Food  дорого   
          freq         54       5   
positive  count      1235    1235   
          unique        5     402   
          top        Food  вкусно   
          freq        484     137   

                                                               text  
sentiment                                                            
negative  count                                                 132  
          unique                                                 91  
          top     Посещали это заведение 2 раза, ощущения разные...  
          freq                                                    4  
positive  count                                                1235  
          unique                                                353  
          top     Отмечали свадьбу 15 марта. Ах-Ах!!! Замечатель...  
          freq                                                   14

In [8]:
# Делаем датасет сбалансированным
df = pd.concat([df[df['sentiment'] == 'positive'].sample(frac=1)[:132], df[df['sentiment'] == 'negative']]).sample(frac=1)

Сделаем датасет сбалансированным


In [10]:
# Загружаем модель
m = 'web_0_300_20.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
model.init_sims(replace=True)

In [104]:
transit = {'ADJF':'ADJ',
'ADJS' : 'ADJ',
'ADVB' : 'ADV',
'COMP' : 'ADV',
'CONJ' : 'CCONJ',
'GRND' : 'VERB',
'INFN' : 'VERB',
'INTJ' : 'INTJ',
'LATN' : 'X',
'NOUN' : 'NOUN',
'NPRO' : 'PRON',
'NUMB' : 'NUM',
'NUMR' : 'NUM',
'PNCT' : 'PUNCT' ,
'PRCL' : 'PART',
'PRED' : 'ADV',
'PREP' : 'ADP',
'PRTF' : 'ADJ',
'PRTS' : 'VERB',
'ROMN' : 'X',
'SYMB' : 'SYM',
'UNKN' : 'X',
'VERB' : 'VERB'}

robj = re.compile('|'.join(transit.keys()))

def cleanization(text):
    for line in text:
        # 1. Все буквы в нижний регистр
        text_text = text.lower()

        # 2. Удаление всех небукв
        letters_only = ''
        for _c in text_text:
            if _c in RUS_LETTERS:
                letters_only += _c
            else:
                letters_only += ' '

        # 3. Заменяем множественные пробелы
        while '  ' in letters_only:
            letters_only = letters_only.replace('  ', ' ')

        # 4. Токенизация
        word_list = tokenizer.tokenize(letters_only)

        # 5. Лемматизация
        clean_word_list = [morph.parse(word)[0].normal_form for word in word_list]  # лемматизация
    
        # 6. * Удаление стоп-слов + добавление тегов - части речи
        # meaningful_words = [word for word in clean_word_list if word not in get_stop_words('ru')] # стоп-слова
        meaningful_words = [str(word) + '_' + robj.sub(lambda m: transit[m.group(0)], str(morph.parse(word)[0].tag.POS)) for word in clean_word_list]
        return ' '.join(meaningful_words) 

In [137]:
# Проверка работы функции cleanization
cleanization('Я сегодня съела очень много мороженого')

'я_PRON сегодня_ADV съесть_VERB очень_ADV много_ADV мороженое_NOUN'

In [167]:
from __future__ import division
def mean(a):
    return sum(a) / len(a)


def word2vec_mean(text):
    """Усредняет вектор слов."""
    arr = []
    clean_text = cleanization(text)
    # для каждого слова в тексте выводим его вектор
    for word in clean_text.split(' '):
        print(word)
        # есть ли слово в модели? Может быть, и нет
        if word in model:
            arr.append(model[word])
    if len(list(map(mean, zip(*arr)))) != 0:
        return list(map(mean, zip(*arr)))
    else:
        return [0 for i in range(0, 300)]



class FunctionFeaturizer(TransformerMixin):
    """ Для создания своего вектора я использовала усредненную векторизацию с помощью word2vec"""
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        fvs = []
        # fvs = word2vec_mean(X)  # если подавать по словам, а не датафрейм
        for datum in X:
            print(datum)
            fv = word2vec_mean(datum)
            fvs.append(fv)
        return np.array(fvs)

In [168]:
# Проверка векторизатора
df1 = pd.DataFrame({'text':['хороший', 'очень хороший'], 'class':['1', '0']})
w2v_featurizer = FunctionFeaturizer() 
x = w2v_featurizer.transform(df1.text)
x

хороший
хороший_ADJ
очень хороший
очень_ADV
хороший_ADJ


array([[  7.33737042e-03,  -2.96452902e-02,   6.50390610e-02,
          3.19109261e-02,   4.67816442e-02,  -6.75016195e-02,
          1.34420572e-02,  -1.00418180e-01,   6.53345138e-02,
          2.46328283e-02,  -7.88142625e-03,  -7.19515756e-02,
         -3.37976255e-02,   2.32609529e-02,   5.84121747e-03,
         -1.20705580e-02,  -3.55095118e-02,   5.70581928e-02,
         -5.28854914e-02,   6.36451971e-03,   4.01164368e-02,
         -4.82101887e-02,  -5.52522391e-02,   1.19763343e-02,
          5.54701425e-02,  -1.48677127e-02,  -1.01240285e-01,
          4.72908318e-02,   4.67002094e-02,   6.18545637e-02,
         -1.20627820e-01,  -9.26931016e-03,   3.15161943e-02,
          2.23013968e-03,  -3.23524722e-03,   3.60459499e-02,
          2.32331306e-02,   4.34190817e-02,  -6.47412986e-02,
          1.71083870e-04,   2.29924582e-02,  -2.39938311e-02,
          3.65550630e-02,   5.90851046e-02,   4.88272235e-02,
         -6.65662438e-02,   1.19906226e-02,   6.42403737e-02,
        

In [169]:
# Делим корпус на тестовый и тернировочный
X_train, X_test, y_train, y_test = train_test_split(df['term'], df['sentiment'], test_size=0.1)

In [170]:
X_train.head()

496         суховато
157    радовали глаз
522        не вкусно
448           уютную
108           вкусно
Name: term, dtype: object

In [171]:
y_train.head()

496    negative
157    positive
522    negative
448    positive
108    positive
Name: sentiment, dtype: object

In [172]:
X_train.describe()

count        237
unique       175
top       вкусно
freq          10
Name: term, dtype: object

In [173]:
def do_smth_with_model(data_train, class_train, data_test, class_test, steps):
    print('\nModel train')
    pipeline = Pipeline(steps=steps)

    cv_results = cross_val_score(pipeline,
                                 data_train,
                                 class_train,
                                 cv=10,
                                 scoring='accuracy',
                                )
    print(cv_results.mean(), cv_results.std())

    pipeline.fit(data_train, class_train)
    class_predicted = pipeline.predict(data_test)
    print(class_predicted)

    print(classification_report(class_test, class_predicted ))

    return pipeline, class_predicted

In [174]:
w2v_featurizer = FunctionFeaturizer()  # создание своего векторизатора

In [152]:
# Свой векторизатор
print('\nCustom Transformer')
lr_pipeline, label_predicted = do_smth_with_model(X_train, y_train,
                                               X_test, y_test, 
                                               steps=[('custom', w2v_featurizer),
                                                      ('classifier', LogisticRegression())])


Custom Transformer

Model train
0.86502173913 0.0790571610235
['positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'negative' 'negative' 'positive' 'positive' 'negative' 'negative'
 'positive' 'positive' 'negative' 'positive' 'positive' 'negative'
 'positive' 'positive' 'negative' 'negative' 'positive' 'negative'
 'positive' 'negative' 'positive']
             precision    recall  f1-score   support

   negative       1.00      0.91      0.95        11
   positive       0.94      1.00      0.97        16

avg / total       0.97      0.96      0.96        27



In [153]:
# Свой векторизатор
print('\nCustom Transformer')
etx_pipeline, label_predicted = do_smth_with_model(X_train, y_train,
                                               X_test, y_test, 
                                               steps=[('custom', w2v_featurizer),
                                                      ('classifier', ExtraTreesClassifier())])


Custom Transformer

Model train
0.81934057971 0.07840661717
['positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'negative' 'negative' 'positive' 'positive' 'negative' 'negative'
 'positive' 'positive' 'negative' 'positive' 'positive' 'negative'
 'positive' 'positive' 'negative' 'negative' 'positive' 'negative'
 'positive' 'negative' 'positive']
             precision    recall  f1-score   support

   negative       1.00      0.91      0.95        11
   positive       0.94      1.00      0.97        16

avg / total       0.97      0.96      0.96        27



## Проверка работы модели на наших тестовых коллокациях

In [228]:
df1 = pd.DataFrame({'text':['отличный выбор', 'не советуем', 'очень советуем', 'очень дорого', 'выше всяких похвал', 'в общем прекрасно', 'нам все понравилось']})
# , 'в целом ничего', 'отвратительный', 'быстро', 'очень плохое обслуживание', 'отличное меню', 'хороший', 'вкусный', 'замечательный', 'приятный', 'красивый', 'отличный'

In [214]:
df1.head()

,text
0,отличный выбор
1,в общем прекрасно


In [215]:
x = w2v_featurizer.transform(df1.text)  # векторизуем наши слова

отличный выбор
отличный_ADJ
выбор_NOUN
в общем прекрасно
в_ADP
общий_ADJ
прекрасно_ADV


In [216]:
x

array([[-0.02267521, -0.02004785, -0.0533559 ,  0.02998884,  0.02433738,
        -0.04528048,  0.02503823, -0.06758565,  0.10700708,  0.11197043,
         0.0055064 , -0.02449975, -0.04704763,  0.08442327, -0.0074172 ,
        -0.05136339, -0.10666844,  0.02820366, -0.02467697,  0.00714388,
        -0.01181722, -0.06201697, -0.09022723,  0.00912218,  0.0017081 ,
         0.00687049, -0.1338036 ,  0.006457  ,  0.0320274 , -0.05980522,
        -0.10798008, -0.03101333, -0.01886609, -0.00393408,  0.00349874,
        -0.03655685, -0.04868337,  0.01883063,  0.01420446, -0.01032911,
         0.04063499,  0.02868253,  0.03263469,  0.07891723,  0.04877253,
        -0.04752337, -0.02643839,  0.0075727 , -0.0302735 , -0.01208394,
        -0.0765805 , -0.07310214, -0.01387591,  0.05331096,  0.02760612,
        -0.0274395 , -0.09663586,  0.00373642,  0.03719021, -0.02005388,
        -0.01238265, -0.04853695, -0.00077484,  0.04640021,  0.01345689,
         0.0307044 , -0.02597226,  0.00235111, -0.0

In [212]:
lr_model = LogisticRegression()
lr_model.fit(w2v_featurizer.transform(X_train), y_train)
cv_results = cross_val_score(lr_model, w2v_featurizer.transform(X_train), y_train, cv=10, scoring='accuracy')
print(cv_results.mean(), cv_results.std())
lr_model.predict(x)

суховато
суховато_ADV
радовали глаз
радовать_VERB
глаз_NOUN
не вкусно
не_PART
вкусно_ADV
уютную
уютный_ADJ
вкусно
вкусно_ADV
не из дешевых
не_PART
из_ADP
дешёвый_ADJ
вежлив
вежливый_ADJ
Вкусное
вкусный_ADJ
внимателен
внимательный_ADJ
приветливые
приветливый_ADJ
забывчивость
забывчивость_NOUN
быстро
быстро_ADV
хамское
хамский_ADJ
оперативно
оперативно_ADV
места мало
место_NOUN
мало_ADV
приветливый
приветливый_ADJ
вкусные
вкусный_ADJ
уютная
уютный_ADJ
любезный
любезный_ADJ
вежливые
вежливый_ADJ
сытные
сытный_ADJ
вкусно
вкусно_ADV
грубых
грубый_ADJ
игнором
игнором_NOUN
доброжелательных
доброжелательный_ADJ
дорогой
дорогой_ADV
не вкусный
не_PART
вкусный_ADJ
чуть теплым
чуть_ADV
тёплый_ADJ
жесткое
жёсткий_ADJ
не дешево
не_PART
дёшево_ADV
невкусные
невкусный_ADJ
претензионная
претензионный_ADJ
суховато
суховато_ADV
доброжелательная
доброжелательный_ADJ
удобно
удобно_ADV
приветливые
приветливый_ADJ
таяло во рту
таять_VERB
в_ADP
рот_NOUN
нерасторопных
нерасторопный_ADJ
чистенько
чистенько_ADV


array(['negative', 'negative'], dtype=object)

In [190]:
df1

,class,text
0,1,отвратительный
1,2,быстро
2,0,очень плохое обслуживание
3,1,отличное меню


In [226]:
def predictor(word, pipeline):
    print(pipeline.predict(word))
    

In [229]:
predictor(df1.text, pipeline)

отличный_ADJ
выбор_NOUN
не_PART
советовать_VERB
очень_ADV
советовать_VERB
очень_ADV
дорого_ADV
выше_ADV
всякий_ADJ
похвала_NOUN
в_ADP
общий_ADJ
прекрасно_ADV
мы_PRON
весь_ADJ
понравиться_VERB
['positive' 'negative' 'negative' 'negative' 'negative' 'negative'
 'positive']
